In [16]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../../ml/"))

from dev_wtp_io_utils import *

In [17]:
# nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
# ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

sys.path.append(os.path.abspath("../../ml/"))
from apply_loocv_and_save import load_and_preprocess
from dev_utils import read_yaml_for_host
import warnings


config_data = read_yaml_for_host("sst_config.yml")



yaml file does not contain host-specific settings for this host. Using default settings.


In [18]:
nonbids_data_path = config_data['nonbids_data_path']
ml_data_folderpath = nonbids_data_path + "fMRI/ml"


In [19]:
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20220818T144138.csv")

In [20]:
include_exclude_list = pd.read_csv("../nsc_subject_exclusions.csv")

In [21]:
test_train_df_raw = test_train_df_raw.merge(include_exclude_list[include_exclude_list.Task=='SST'],left_on='sub_label',right_on='SubjectId',how='left')
test_train_df_raw.loc[test_train_df_raw.Include.isna(),'Include'] = True
test_train_df = test_train_df_raw[test_train_df_raw.Include==True]

In [22]:
exclude_subjects = ['DEV061','DEV185','DEV187','DEV189','DEV190','DEV192','DEV198','DEV203','DEV220','DEV221']
#2022-01-06 added DEV157 because of an error

In [23]:
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()#only get the train subjects; ignore those previously marked hold-out

In [24]:
def get_Brain_Data_for_sub(sub_label,behavdesign,betaseries_path='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/'):
    return(dev_wtp_io_utils.get_Brain_Data_betas_for_sub(sub_label,behavdesign,betaseries_path))

In [25]:
from dev_wtp_io_utils import import_sst_betaseries_w1_subjs_to_pkl

In [26]:
all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")
#cut out other waves
sst_1_behavdesign_1 = all_sst_events[(all_sst_events.wave=='wave1')].reset_index(drop=True)
#label the ITI as such
sst_1_behavdesign_1.loc[(all_sst_events.trial_type.isna()),'trial_type']='ITI'
#ensure it is correctly ordered
sst_1_behavdesign_clean = sst_1_behavdesign_1.sort_values(['subject','onset']).reset_index(drop=True)
#now tell it the name of the corresponding beta. this is easy; they are all consecutive
sst_1_behavdesign_clean['beta'] = 'beta_' + (sst_1_behavdesign_clean.groupby(['subject','wave']).cumcount()+1).astype(str).apply(lambda x: x.zfill(4)) + '.nii'

In [27]:
#I want to import this other module; we don't actually use it but it's a good reference
#for future programmers to see
#it's important we keep the logic consistent for the two main conditions
sys.path.append(os.path.abspath("../../fx/multiconds/SST/"))
from multiconds import create_posterror_masks_from_masks

#sst_1_behavdesign_clean['prev_trial_type'] = sst_1_behavdesign_clean['trial_type']
sst_1_behavdesign_clean['prev_trial_type'] = (
    sst_1_behavdesign_clean
    .loc[sst_1_behavdesign_clean['trial_type']!='ITI']
    .groupby(['subject','wave'])
    .shift(1)
    ['trial_type']
)#this works because during assignment, items will be assigned to the line corresponding to their index
#very important you don't reset index during this operation.
is_post_stop_correct_go = (
    (sst_1_behavdesign_clean['trial_type']=='correct-go') & 
    (sst_1_behavdesign_clean['prev_trial_type'].isin(['failed-stop','correct-stop']))
)

sst_1_behavdesign_clean.loc[is_post_stop_correct_go,'post_stop_correct_go_subtype'] = 'Other'
sst_1_behavdesign_clean.loc[is_post_stop_correct_go & (sst_1_behavdesign_clean['prev_trial_type']=='failed-stop'),'post_stop_correct_go_subtype'] = 'post-error'
sst_1_behavdesign_clean.loc[is_post_stop_correct_go & (sst_1_behavdesign_clean['prev_trial_type']=='correct-stop'),'post_stop_correct_go_subtype'] = 'post-correct'


In [28]:
sst_1_behavdesign_clean.groupby(['trial_type','post_stop_correct_go_subtype']).size().reset_index()

,trial_type,post_stop_correct_go_subtype,0
0,correct-go,post-correct,1879
1,correct-go,post-error,2145


In [29]:
sst_1_behavdesign_clean['trial_type'].value_counts()

ITI             26112
correct-go      18750
failed-stop      3434
correct-stop     3094
failed-go         834
Name: trial_type, dtype: int64

In [30]:
behavdesign = sst_1_behavdesign_clean
sst_1_behavdesign_clean_dev005 = behavdesign[[s in 'DEV005' for s in behavdesign.subject]]

## Just subjects in the conditions of interest: CorrectStop vs. CorrectGo

In [31]:
sst_1_behavdesign_clean.trial_type.value_counts()

ITI             26112
correct-go      18750
failed-stop      3434
correct-stop     3094
failed-go         834
Name: trial_type, dtype: int64

In [32]:
sst_1_behavdesign_correct_conditions=sst_1_behavdesign_clean[sst_1_behavdesign_clean.trial_type.isin(['correct-go','correct-stop'])]

In [33]:
sst_1_behavdesign_correct_conditions.trial_type.value_counts()

correct-go      18750
correct-stop     3094
Name: trial_type, dtype: int64

In [34]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq


train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [35]:
sst_1_behavdesign_correct_conditions[sst_1_behavdesign_correct_conditions.subject=='DEV157']

,onset,duration,trial_type,subject,wave,beta,prev_trial_type,post_stop_correct_go_subtype
37376,0.00000,0.87197,correct-go,DEV157,wave1,beta_0001.nii,NaN,NaN
37380,6.38890,0.08329,correct-go,DEV157,wave1,beta_0005.nii,failed-stop,post-error
37384,15.65280,0.59839,correct-go,DEV157,wave1,beta_0009.nii,failed-stop,post-error
37386,20.53475,0.46574,correct-go,DEV157,wave1,beta_0011.nii,correct-go,NaN
37388,23.35212,0.34208,correct-go,DEV157,wave1,beta_0013.nii,correct-go,NaN
...,...,...,...,...,...,...,...,...
37618,400.61111,0.43916,correct-go,DEV157,wave1,beta_0243.nii,correct-go,NaN
37620,404.11667,0.72024,correct-go,DEV157,wave1,beta_0245.nii,correct-go,NaN
37622,407.24723,0.43989,correct-go,DEV157,wave1,beta_0247.nii,correct-go,NaN
37626,412.44516,0.00170,correct-go,DEV157,wave1,beta_0251.nii,failed-stop,post-error


### Now mask those subjects

In [36]:
# get the PFC mask
mask_pfc = nib.load(ml_data_folderpath + '/prefrontal_cortex.nii.gz')

In [ ]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:6],'betaseries',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    mask = mask_pfc,
    out_file_suffix = '_correct_cond',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/")

In [ ]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:30],'betaseries',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    mask = mask_pfc,
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/")

In [37]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs,'betaseries',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    mask = mask_pfc,
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/")

running
really
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/
DEV005
........................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0001.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
         onset  duration    trial_type subject   wave           beta  \
256    0.00000   2.25834  correct-stop  DEV005  wave1  beta_0001.nii   
258    2.75834   0.40082    correct-go  DEV005  wave1  beta_0003.nii   
260    5.51390   0.66191    correct-go  DEV005  wave1  beta_0005.nii   
264   12.52780   0.51712    correct-go  DEV005  wave1  beta_0009.nii   
266   15.90975   0.39906    correct-go  DEV005  wave1  beta_0011.nii   
..         ...       ...           ...     ...    ...            ...   
498  398.86111   0.49

..................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0005.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration  trial_type subject   wave           beta  \
2052    6.38890   0.88393  correct-go  DEV012  wave1  beta_0005.nii   
2056   15.65280   0.70575  correct-go  DEV012  wave1  beta_0009.nii   
2058   20.53475   0.55686  correct-go  DEV012  wave1  beta_0011.nii   
2060   23.35212   0.57174  correct-go  DEV012  wave1  beta_0013.nii   
2062   25.98407   0.37049  correct-go  DEV012  wave1  beta_0015.nii   
...         ...       ...         ...     ...    ...            ...   
2290  400.61111   0.49393  correct-go  DEV012  wave1  beta_0243.nii   
2292  404.11667   0.60228  correct-go  DEV012  wave1

......................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
3330    3.00834   2.00556  correct-stop  DEV017  wave1  beta_0003.nii   
3332    6.38890   0.56791    correct-go  DEV017  wave1  beta_0005.nii   
3336   15.65280   0.56373    correct-go  DEV017  wave1  beta_0009.nii   
3338   20.53475   0.45915    correct-go  DEV017  wave1  beta_0011.nii   
3340   23.35212   0.37217    correct-go  DEV017  wave1  beta_0013.nii   
...         ...       ...           ...     ...    ...            ...   
3572  404.11667   0.46661    correct-go  DEV017  wave1  beta_0245.nii   
3574  407.24723   0.52759    cor

......................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0005.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
4612    6.38890   0.71152    correct-go  DEV022  wave1  beta_0005.nii   
4616   15.65280   0.61978    correct-go  DEV022  wave1  beta_0009.nii   
4618   20.53475   0.61503    correct-go  DEV022  wave1  beta_0011.nii   
4620   23.35212   0.59552    correct-go  DEV022  wave1  beta_0013.nii   
4622   25.98407   0.58036    correct-go  DEV022  wave1  beta_0015.nii   
...         ...       ...           ...     ...    ...            ...   
4850  400.61111   0.53491    correct-go  DEV022  wave1  beta_0243.nii   
4852  404.11667   0.91152    cor

.....................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
          onset  duration    trial_type subject   wave           beta  \
5890    3.00834   2.00556  correct-stop  DEV027  wave1  beta_0003.nii   
5892    6.38890   0.88834    correct-go  DEV027  wave1  beta_0005.nii   
5894   11.77224   2.00556  correct-stop  DEV027  wave1  beta_0007.nii   
5896   15.65280   0.69325    correct-go  DEV027  wave1  beta_0009.nii   
5898   20.53475   0.71644    correct-go  DEV027  wave1  beta_0011.nii   
...         ...       ...           ...     ...    ...            ...   
6132  404.11667   0.45231    correct-go  DEV027  wave1  beta_0245.nii   
6134  407.24723  

...

..............................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0003.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
           onset  duration    trial_type subject   wave           beta  \
11522    3.00834   2.00556  correct-stop  DEV049  wave1  beta_0003.nii   
11524    6.38890   0.71550    correct-go  DEV049  wave1  beta_0005.nii   
11526   11.77224   2.00556  correct-stop  DEV049  wave1  beta_0007.nii   
11528   15.65280   0.65665    correct-go  DEV049  wave1  beta_0009.nii   
11532   23.35212   0.72633    correct-go  DEV049  wave1  beta_0013.nii   
...          ...       ...           ...     ...    ...            ...   
11758  391.47499   0.87907    correct-go  DEV049  wave1  beta_0239.nii   
11766  407.24723   0.56429    co

converting to nifti and fdata
           onset  duration    trial_type subject   wave           beta  \
12546    2.75834   0.76828    correct-go  DEV053  wave1  beta_0003.nii   
12548    5.51390   0.43770    correct-go  DEV053  wave1  beta_0005.nii   
12550    9.02224   0.40417    correct-go  DEV053  wave1  beta_0007.nii   
12552   12.52780   0.38164    correct-go  DEV053  wave1  beta_0009.nii   
12554   15.90975   0.41218    correct-go  DEV053  wave1  beta_0011.nii   
...          ...       ...           ...     ...    ...            ...   
12790  403.99723   0.54811    correct-go  DEV053  wave1  beta_0247.nii   
12792  407.31460   0.53765    correct-go  DEV053  wave1  beta_0249.nii   
12794  410.82016   0.42981    correct-go  DEV053  wave1  beta_0251.nii   
12796  414.57572   0.57988    correct-go  DEV053  wave1  beta_0253.nii   
12798  417.83682   1.50000  correct-stop  DEV053  wave1  beta_0255.nii   

      prev_trial_type post_stop_correct_go_subtype  
12546     failed-stop       

....................................................................................................................verified that all expected betas exist! Processing...retrieving a mask from the first image in the series beta_0005.nii
...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
           onset  duration    trial_type subject   wave           beta  \
14084    6.38890   0.59165    correct-go  DEV059  wave1  beta_0005.nii   
14088   15.65280   0.54624    correct-go  DEV059  wave1  beta_0009.nii   
14090   20.53475   0.58688    correct-go  DEV059  wave1  beta_0011.nii   
14092   23.35212   0.50494    correct-go  DEV059  wave1  beta_0013.nii   
14094   25.98407   0.42816    correct-go  DEV059  wave1  beta_0015.nii   
...          ...       ...           ...     ...    ...            ...   
14326  407.24723   0.40694    correct-go  DEV059  wave1  beta_0247.nii   
14328  409

...using a combined mask of the first image and the passed in masktemp mask created.
loading files. This step may take some time...
converting to nifti and fdata
           onset  duration    trial_type subject   wave           beta  \
15878   11.77224   2.00556  correct-stop  DEV067  wave1  beta_0007.nii   
15882   20.53475   0.98298    correct-go  DEV067  wave1  beta_0011.nii   
15886   25.98407   0.00262    correct-go  DEV067  wave1  beta_0015.nii   
15888   28.92644   0.81914    correct-go  DEV067  wave1  beta_0017.nii   
15890   31.43061   0.77348    correct-go  DEV067  wave1  beta_0019.nii   
...          ...       ...           ...     ...    ...            ...   
16116  404.11667   0.82027    correct-go  DEV067  wave1  beta_0245.nii   
16118  407.24723   0.97949    correct-go  DEV067  wave1  beta_0247.nii   
16120  409.68960   2.00556  correct-stop  DEV067  wave1  beta_0249.nii   
16124  416.20072   2.51110  correct-stop  DEV067  wave1  beta_0253.nii   
16126  421.21182   0.842

Exception: beta beta_0003.nii does not exist at /gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/sub-DEV068/beta_0003.nii

In [ ]:
print ("running")
print('really')
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs,'betaseries',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
    mask = mask_pfc,
    out_file_suffix = '_correct_cond_pfc',
    sst_wt_repo = nonbids_data_path + "fMRI/fx/models/SST/wave1/")